# NICE2019: BrainScaleS-2 Tutorial 2019-03-29


<img src="img/hicann_neurons_cutout.png" style="width:100%" />

Main contributors:
* Phillip Spilger <img src="img/psp.jpg" style="width:20%" />
* Timo Wunderlich <img src="img/timow.jpg" style="width:20%" />

## Common features of the BrainScaleS architectures:

* **Analog** implementation
* **Physical model**: Neurons and synapses realized on CMOS
* **Time-continuous**/non-multiplexed
* Spikes events are digital & asynchronous
* Configurable (model parameters, connectivity/topology)
* **Accelerated** model (10³ - 10⁴ times faster compared to biological model time constants)



## HICANN-DLS v2 Prototype

For this tutorial we will use a small prototype containing 32 neurons (LIF), and 1024 synapses in total.
The acceleration factor is approx. 1000 compared to the biological model.
An embedded SIMD processor extends the possiblilites compares to BrainScaleS-1 and other analog systems.

<img src="img/IMG_0038.JPG" style="width:30%" />

* Targeting flexible plasticity rules and on-the-fly continuous reconfiguration (modifications of STDP learning rule, structural plasticity)
* Offload software tasks to embedded (plasticity) processor (PPU):
  * Supervised learning
  * External rewards
  * Reinforcement learning
  * Neuronal plasticity (calibration)
  * Anything you can code…
  * At any time scale (bio-ms to bio-years)
  * Scalability and standalone operation


## Latest BrainScaleS-2 chip: HICANN-X

(Cf. Johannes talk)

<img src="img/HICANNX_small.jpg" style="width:30%" />

* Structured neurons
* "ANN"-mode

## Embedded processor dedicated to programmable plasticity?

### Observables:

* Per-synapse pre-post correlation measurements <img src="img/friedmann_correlation.png" width="50%" />
* Neuron spike counts
* Spike events (in/out)

### Controllables:

* Weights (& connectivity)
* Neuron parameters & structure, . . .

### Allows for

* (Two-factor) and reward-modulated STDP
* Structural plasticity
* Homeostasis
* Other, phenomenological plasticity rules

### What else?

* Interaction with virtual environments <img src="img/paper_timo_expoverview.png" width="50%" />
* Motor/sensor loops
* *“Model debugging”, e.g. snapshotting some state variables*

## Demonstrator Videos

* Local Learning Demo on BrainScaleS-2 https://youtu.be/LW0Y5SSIQU4
* Unsupervised learning Demo on BrainScaleS-2 https://youtu.be/x3l1xl8orhQ

# Tutorial Overview

This notebook serves as a hands-on tutorial to the HICANN-DLS v2 prototype chip. Later full-sized chips, such as HICANN-X, are in many aspects very similar to this prototype.

The following schematic shows the software layers. This tutorial covers the `haldls` and `stadls` layers.

<img src="img/bss2_software_stack_modified.png" width="30%" />

We will learn:
* how to create an initial configuration (neuron & synapse parameters, technical settings)
* how to design an experiment *protocol* (→ timed sequence of input spikes)
* how to upload and run the configuration and experiment protocol
* how to download the result data and plot it

## Stage 0: Single neuron stimulation

* generating stimulation data
* configure single neuron
* stimulate with spike train
* read back spike data
* plot input and output spikes

### Stimulation / Spike Train
The following function simply returns a "Poisson spike train":

```python
import numpy
def generate_poisson_spiketrain(isi, time_interval):
    dts = numpy.random.exponential(isi, size=time_interval / isi)
    spikes = numpy.add.accumulate(dts, dtype=int)
    return spikes
```

BrainScaleS systems use a mixture of circuit switching and tag/label filtering to address synapses. In order to send a spike train to a certain synapse we have to add "target synapse driver" (→ target row in the synapse array) and "label" information (synapses have a configurable filter on labels):

In [ ]:
import numpy
from dlens.v2 import hal # PlaybackSpike

def generate_poisson_spiketrain(isi, timeframe, label, synapse_driver):
    dts = numpy.random.exponential(isi, size=timeframe / isi)
    ts = numpy.add.accumulate(dts, dtype=int)
    spikes = []
    for t in ts:
        spikes.append(hal.PlaybackSpike(t, label, synapse_driver))
    return spikes

### Generate spike trains:

In [ ]:
from dlens.v2 import halco # SynapseDriverOnDLS

# inter-spike interval
noise_isi = 1000

# time interval of spikes (implicitly start from t=0)
timeframe = 100000

# target synapse driver (row)
target_synapse_driver = halco.SynapseDriverOnDLS(23)

# target synapse label
target_synapse_label1 = 42
target_synapse_label2 = 43

spike_train1 = generate_poisson_spiketrain(noise_isi, timeframe,
                                          target_synapse_label1, target_synapse_driver)
spike_train2 = generate_poisson_spiketrain(noise_isi, timeframe,
                                          target_synapse_label2, target_synapse_driver)

spike_train1.sort()
spike_train2.sort()

### Plotting:

In [ ]:
%matplotlib notebook
import numpy, math
from matplotlib import pyplot as plt

plt.eventplot([spike.time for spike in spike_train1], color=['red'],
              lineoffsets=[target_synapse_label1], label="Spike Train 1")
plt.eventplot([spike.time for spike in spike_train2], color=['blue'],
              lineoffsets=[target_synapse_label2], label="Spike Train 2")

plt.yticks(range(int(math.ceil(min(plt.yticks()[0]))), int(math.ceil(max(plt.yticks()[0])))))
plt.title("DLSv2 single neuron spike measurement")
plt.xlabel("Spike Time [hardware cycles, 10ns]")
plt.ylabel("Spike Identifier")
plt.legend()

### Specify the individual hardware system

In [ ]:
# assign an DLS Setup to each jupyter user (based on userid)
import os
userid = int(os.environ['HOME'].split('/')[-1])
print("My user id: {}".format(userid))
DLSSetups = ['B291656', 'B201330'] #, '07', 'B291698'] # available systems
DLSSetup = DLSSetups[userid % len(DLSSetups)]
print("This notebook will use Setup {}".format(DLSSetup))
os.environ['USER']='s1ext_user1'

### Stage 0 Experiment: Configure hardware, provide input data, run emulation and read back data

We hide the default setup code in tutorial_configuration.py (if interested: open via Jupyter's File dialog).

In [ ]:
%%writefile stage0.py

from dlens import logger
from dlens.v2 import hal, sta, halco
from dlens.v2.halco import NeuronOnDLS, NeuronParameter, SynapseOnDLS

from tutorial_configuration import TutorialBoard, TutorialChip

import os
BOARD_ID = os.environ["QUIGGELDY_BOARD"]

logger.default_config(level=logger.LogLevel.INFO)

import numpy

def generate_poisson_spiketrain(isi, timeframe, address, synapse_driver):
    dts = numpy.random.exponential(isi, size=timeframe / isi)
    ts = numpy.add.accumulate(dts, dtype=int)
    spikes = []
    for t in ts:
        spikes.append(hal.PlaybackSpike(t, address, synapse_driver))
    return spikes

class SingleNeuronSpikes:
    def __init__(self):
        self.ctrl = sta.ExperimentControl()
        self.board = TutorialBoard()
        self.chip = TutorialChip()

    def execute(self, leak, nrn, noise_isi = 1000, timeframe = 1000000, excitatory_weight = 63):
        """
        This script emplaces excitatory Poisson noise onto a neuron via one synapse and
        measures the neuron spikes.

        :param leak: Leak potential to set prior to sending the spiketrain
        :type leak: hal.DAC.Value
        :param nrn: Neuron to stimulate
        :type nrn: NeuronOnDLS coordinate
        :param noise_isi: Mean inter-spike-interval of Poisson spike sources
        :type noise_isi: int
        :param timeframe: Timeframe of spiketrains
        :type timeframe: int
        :param excitatory_weight: Weight of synaptic connection to excitatory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :return: Spikes sent and spikes received
        :rtype: list[list[hal.PlaybackSpike], list[hal.RecordedSpike]]
        """

        # Logger object
        log = logger.get("execute")

        # Set leak potential
        for nrn in halco.iter_all(NeuronOnDLS):
            self.chip.capmem.set(nrn, NeuronParameter.v_leak, leak)

        synapse_address = hal.SynapseBlock.Synapse.Address(42) # arbitrary != 0

        # Configure excitatory background synapses
        synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                 self.chip.get_excitatory_synapse_driver().toSynapseRowOnDLS())
        synapse = self.chip.get_synapse(synapse_coord)
        synapse.weight = excitatory_weight
        synapse.address = synapse_address
        self.chip.set_synapse(synapse_coord, synapse)

        # Create a playback program (all times are in FPGA cycles / 96MHz)
        builder = hal.PlaybackProgramBuilder()

        # Create excitatory and Poisson spiketrain
        input_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_excitatory_synapse_driver())
        input_spikes.sort()

        # Add spikes to playback program
        builder.set_time(0)
        for spike in input_spikes:
            builder.wait_until(spike.time)
            builder.fire(spike.synapse_driver, spike.source_address)
        builder.halt()
        program = builder.done()
        assert isinstance(program, hal.PlaybackProgram)

        self.ctrl.run_experiment(self.board, self.chip, program)

        neuron_spikes = program.get_spikes()
        log.TRACE("Received spikes: %s" % neuron_spikes)

        return [input_spikes, neuron_spikes]


if __name__ == '__main__':
    experiment = SingleNeuronSpikes()

    neuron = NeuronOnDLS(1)
    #neuron = NeuronOnDLS(17)
    #neuron = NeuronOnDLS(4)
    input_spikes, neuron_spikes = experiment.execute(leak=400, noise_isi=200, nrn=neuron, timeframe=20000)

    input_spike_times = [spike.time for spike in input_spikes]
    neuron_spike_times = [spike.time for spike in neuron_spikes]
    
    #print len(neuron_spike_times)
    #print len(input_spike_times)
    
    numpy.save('neuron_spike_times.data', numpy.array(neuron_spike_times))
    numpy.save('input_spike_times.data', numpy.array(input_spike_times))


### Submit the experiment into the queuing system

In [ ]:
!srun -p dls --daas={DLSSetup} -- singularity exec --app visionary-dls /containers/stable/latest python stage0.py
!ls *spike_times.data*

### Plotting

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy, math

neuron_spike_times = numpy.load('neuron_spike_times.data.npy')
input_spike_times = numpy.load('input_spike_times.data.npy')

plt.eventplot([input_spike_times], color=['blue'], lineoffsets=[1], label="Input spikes")
plt.eventplot([neuron_spike_times], color=['red'], lineoffsets=[0], label="Neuron output spikes")
plt.yticks(range(int(math.ceil(min(plt.yticks()[0]))), int(math.ceil(max(plt.yticks()[0])))))
plt.title("DLSv2 single neuron spike measurement")
plt.xlabel("Spike Time [hardware cycles, 10ns]")
plt.ylabel("Spike Identifier")
plt.legend()

## Stage 0b: Decreasing weights over time

We can use the embedded processor to change model parameters during experiment runtime.

* Decrease weights periodically (with weight-wrap-around) until experiment end time is reached.

In [ ]:
%%writefile spike_counter_loop_decrease_weight_over_time.cc

#include <stddef.h>
#include <stdint.h>
extern "C"
{
#include "libnux/counter.h"
#include "libnux/dls.h"
#include "libnux/spr.h"
#include "libnux/syn.h"
#include "libnux/time.h"
}

extern uint8_t ram_end;

// This program reads out neuron-local spike counters in a loop. The per-neuron accumulation is
// stored to a defined memory address range after reading for a defined timeframe for readout from
// the host.

// Program entry point
extern "C" int start()
{
	vector uint8_t excitatory_weights_l;
	vector uint8_t excitatory_weights_r;

	uint8_t excitatory_row = 0;

	// Timeframe of spike count measurement
	time_base_t constexpr timeframe = 2000000;

	// Sleep period [cycles] between consecutive readouts
	uint32_t constexpr sleep_period = 10000;
    
    // read in initial weights
	get_weights(&excitatory_weights_l, &excitatory_weights_r, excitatory_row);

	// decrease weights over time (w/ wrap around)
	while (get_time_base() < timeframe) {

		excitatory_weights_l = excitatory_weights_l - vec_splat_u8(1);
		excitatory_weights_r = excitatory_weights_r - vec_splat_u8(1);

		set_weights(&excitatory_weights_l, &excitatory_weights_r, excitatory_row);

		sleep_cycles(sleep_period);
	}

	return 0;
}


### Cross-compiling of code for embedded processor

We use a build tool for compilation of host code as well as PPU code. For demonstration, we perform the individual steps explicitly in this tutorial.

The first step: Compile the source into an object file.

In [ ]:
!powerpc-ppu-g++ -std=gnu++14 -fdiagnostics-color=always -O2 -fno-strict-aliasing -Wall -Wextra -pedantic -ffreestanding -mcpu=nux -std=gnu++11 -fno-exceptions -fno-rtti -fno-non-call-exceptions -fno-common -ffunction-sections -fdata-sections -Ilibnux -Ilibnux -Idemo-dls/src/ppu -Idemo-dls/src/ppu -DLIBNUX_DLS_VERSION=2 -DNDEBUG -DSYSTEM_HICANN_DLS_MINI spike_counter_loop_decrease_weight_over_time.cc -c -ospike_counter_loop_decrease_weight_over_time.cc.7.o

The second step: Link the object file and helper libraries (there is no full runtime available on the PPU).

In [ ]:
!powerpc-ppu-g++ -fuse-ld=bfd -nostdlib -Lbuild/libnux -Tlibnux/libnux/elf32nux.x -Wl,--gc-sections spike_counter_loop_decrease_weight_over_time.cc.7.o build/libnux/libnux/counter.c.10.o build/libnux/libnux/time.c.9.o build/libnux/src/crt.s.6.o build/libnux/src/cxa_pure_virtual.c.4.o build/libnux/src/initdeinit.c.3.o -ofirmware/spike_counter_loop_decrease_weight_over_time.bin -Bstatic -Llibnux -lgcc -lnux -Bdynamic

The last step: Strip down the output file to the binary code.

In [ ]:
!powerpc-ppu-objcopy -O binary firmware/spike_counter_loop_decrease_weight_over_time.bin firmware/spike_counter_loop_decrease_weight_over_time.binary

### Experiment description (part not running on the embedded processor)

The host experiment script is very similar to the previous one. The main difference is the specification of a executable for the embedded processor

In [ ]:
%%writefile stage0b.py

from dlens import logger
from dlens.v2 import hal, sta, halco
from dlens.v2.halco import NeuronOnDLS, NeuronParameter, SynapseOnDLS

from tutorial_configuration import TutorialBoard, TutorialChip

logger.default_config(level=logger.LogLevel.INFO)

import numpy

def generate_poisson_spiketrain(isi, timeframe, address, synapse_driver):
    dts = numpy.random.exponential(isi, size=timeframe / isi)
    ts = numpy.add.accumulate(dts, dtype=int)
    spikes = []
    for t in ts:
        spikes.append(hal.PlaybackSpike(t, address, synapse_driver))
    return spikes

class SingleNeuronSpikes:
    def __init__(self):
        self.ctrl = sta.ExperimentControl()
        self.board = TutorialBoard()
        self.chip = TutorialChip()

    def execute(self, leak, nrn, noise_isi = 1000, timeframe = 1000000,
                excitatory_weight = 63):
        """
        This script emplaces excitatory Poisson noise onto a neuron via one synapse and
        measures the neuron spikes.

        :param leak: Leak potential to set prior to sending the spiketrain
        :type leak: hal.DAC.Value
        :param nrn: Neuron to stimulate
        :type nrn: NeuronOnDLS coordinate
        :param noise_isi: Mean inter-spike-interval of Poisson spike sources
        :type noise_isi: int
        :param timeframe: Timeframe of spiketrains
        :type timeframe: int
        :param excitatory_weight: Weight of synaptic connection to excitatory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :return: Spikes sent and spikes received
        :rtype: list[list[hal.PlaybackSpike], list[hal.RecordedSpike]]
        """

        # Logger object
        log = logger.get("execute")

        # Set leak potential
        for nrn in halco.iter_all(NeuronOnDLS):
            self.chip.capmem.set(nrn, NeuronParameter.v_leak, leak)

        synapse_address = hal.SynapseBlock.Synapse.Address(42) # arbitrary != 0

        # Configure excitatory background synapses
        synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                 self.chip.get_excitatory_synapse_driver().toSynapseRowOnDLS())
        synapse = self.chip.get_synapse(synapse_coord)
        synapse.weight = excitatory_weight
        synapse.address = synapse_address
        self.chip.set_synapse(synapse_coord, synapse)

        # Create a playback program (all times are in FPGA cycles / 96MHz)
        builder = hal.PlaybackProgramBuilder()

        # Create excitatory and Poisson spiketrain
        input_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_excitatory_synapse_driver())
        input_spikes.sort()

        # Create PPU (Plasticity Processing Unit) memory object and load pragram
        ppu_memory = hal.PPUMemory()
        ppu_memory.load_from_file("firmware/spike_counter_loop_decrease_weight_over_time.binary")

        # Pull PPU-reset
        ppu_control_register = hal.PPUControlRegister()
        ppu_control_register.inhibit_reset = False
        builder.write(halco.PPUControlRegisterOnDLS(), ppu_control_register)

        # Write PPU memory
        builder.write(halco.PPUMemoryOnDLS(), ppu_memory)

        # Release PPU-reset
        ppu_control_register.inhibit_reset = True
        builder.write(halco.PPUControlRegisterOnDLS(), ppu_control_register)

        # Add spikes to playback program
        builder.set_time(0)
        for spike in input_spikes:
            builder.wait_until(spike.time)
            builder.fire(spike.synapse_driver, spike.source_address)
        builder.halt()
        program = builder.done()
        assert isinstance(program, hal.PlaybackProgram)

        self.ctrl.run_experiment(self.board, self.chip, program)

        neuron_spikes = program.get_spikes()
        log.TRACE("Received spikes: %s" % neuron_spikes)

        return [input_spikes, neuron_spikes]


if __name__ == '__main__':
    experiment = SingleNeuronSpikes()

    neuron = NeuronOnDLS(1)
    #neuron = NeuronOnDLS(17)
    #neuron = NeuronOnDLS(4)
    input_spikes, neuron_spikes = \
        experiment.execute(leak=400, noise_isi=500, nrn=neuron, timeframe=2000000)

    input_spike_times = [spike.time for spike in input_spikes]
    neuron_spike_times = [spike.time for spike in neuron_spikes]
    
    #print len(neuron_spike_times)
    #print len(input_spike_times)
    
    numpy.save('neuron_spike_times.data', numpy.array(neuron_spike_times))
    numpy.save('input_spike_times.data', numpy.array(input_spike_times))


### Submit the experiment into the queuing system

In [ ]:
!srun -p dls --daas={DLSSetup} -- singularity exec --app visionary-dls /containers/stable/latest python stage0b.py
!ls *spike_times.data*

### Plotting

In [ ]:
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy, math

neuron_spike_times = numpy.load('neuron_spike_times.data.npy')
input_spike_times = numpy.load('input_spike_times.data.npy')

plt.eventplot([input_spike_times], color=['blue'], lineoffsets=[1], label="Input spikes")
plt.eventplot([neuron_spike_times], color=['red'], lineoffsets=[0], label="Neuron output spikes")
plt.yticks(range(int(math.ceil(min(plt.yticks()[0]))), int(math.ceil(max(plt.yticks()[0])))))
plt.title("DLSv2 single neuron spike measurement")
plt.xlabel("Spike Time [hardware cycles, 10ns]")
plt.ylabel("Spike Identifier")
plt.legend()

## Stage 1: Recording neuronal activation functions

* Neurons receive balanced Poisson noise.
* Measure "spiking probability" as function of resting membrane potential.
  

## 1.1: Use Host to read spikes from FPGA.

* Spikes from the chip are buffered in the FPGA.
* Read spikes from FPGA and count them at different resting potentials.

Class `NeuronActivation` encapsulates the board & chip configuration and is used to run the experiment and collect output spikes.
All times are given in units of FPGA cycles (T = 10.2 nanoseconds).

In [ ]:
%%writefile stage1_host_file1.py

import numpy
from tutorial_configuration import TutorialBoard, TutorialChip

from dlens import logger
from dlens.v2 import hal, sta, halco
from dlens.v2.halco import NeuronOnDLS, NeuronParameter, SynapseOnDLS

#logger.default_config(level=logger.LogLevel.INFO)
logger.default_config(level=logger.LogLevel.ERROR)

def generate_poisson_spiketrain(isi, timeframe, address, synapse_driver):
    dts = numpy.random.exponential(isi, size=timeframe / isi)
    ts = numpy.add.accumulate(dts, dtype=int)
    spikes = []
    for t in ts:
        spikes.append(hal.PlaybackSpike(t, address, synapse_driver))
    return spikes

class NeuronActivation:
    def __init__(self):
        self.ctrl = sta.ExperimentControl()
        self.board = TutorialBoard()
        self.chip = TutorialChip()

    def execute(self, leak, noise_isi = 1000, timeframe = 1000000, excitatory_weight = 50, inhibitory_weight = 50):
        """
        This script measures the neuron activation of all neurons in parallel by sending excitatory and
        inhibitory Poisson spiketrains and measuring the number of neuron spikes per neuron.

        :param leak: Leak potential to set prior to sending the spiketrains
        :type leak: hal.DAC.Value
        :param noise_isi: Mean inter-spike-interval of Poisson spike sources
        :type noise_isi: int
        :param timeframe: Timeframe of spiketrains
        :type timeframe: int
        :param excitatory_weight: Weight of synaptic connection to excitatory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :param inhibitory_weight: Weight of synaptic connection to inhibitory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :return: Spikes received
        :rtype: list[hal.RecordedSpike]
        """

        # Set leak potential
        for nrn in halco.iter_all(NeuronOnDLS):
            self.chip.capmem.set(nrn, NeuronParameter.v_leak, leak)

        synapse_address = hal.SynapseBlock.Synapse.Address(42) # arbitrary != 0

        # Configure excitatory background synapses
        for nrn in halco.iter_all(NeuronOnDLS):
            synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                     self.chip.get_excitatory_synapse_driver().toSynapseRowOnDLS())
            synapse = self.chip.get_synapse(synapse_coord)
            synapse.weight = excitatory_weight
            synapse.address = synapse_address
            self.chip.set_synapse(synapse_coord, synapse)

        # Configure inhibitory background synapses
        for nrn in halco.iter_all(NeuronOnDLS):
            synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                     self.chip.get_inhibitory_synapse_driver().toSynapseRowOnDLS())
            synapse = self.chip.get_synapse(synapse_coord)
            synapse.weight = inhibitory_weight
            synapse.address = synapse_address
            self.chip.set_synapse(synapse_coord, synapse)

        # Create a playback program (all times are in FPGA cycles / 96MHz)
        builder = hal.PlaybackProgramBuilder()

        # Create excitatory and inhibitory poisson spiketrains
        excitatory_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_excitatory_synapse_driver())
        inhibitory_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_inhibitory_synapse_driver())
        all_spikes = excitatory_spikes + inhibitory_spikes
        all_spikes.sort()

        # Add spikes to playback program
        builder.set_time(0)
        for spike in all_spikes:
            builder.wait_until(spike.time)
            builder.fire(spike.synapse_driver, spike.source_address)
        builder.halt()
        program = builder.done()
        assert isinstance(program, hal.PlaybackProgram)

        self.ctrl.run_experiment(self.board, self.chip, program)

        spikes = program.get_spikes()
        return spikes


The following code section uses `NeuronActivation`  to sweep the neurons' `leak` parameter.
The  membrane leak potential DAC value (12-bit digital value) is typically set in the range (300, 700).
We can use `NeuronActivation.execute(leak=X, ...)` to perform an experiment with a given `leak` and retrieve the output of the experiment (spikes).

In [ ]:
%%writefile stage1_host_file2.py

neuron_activation = NeuronActivation()

nrn_spike_counts = []

#dac_value = 300
#spikes = neuron_activation.execute(leak=dac_value, timeframe=2000000)
#nrn_spike_counts.append([dac_value, len(spikes)])

# Execute activation measurement for a leak potential sweep
for dac_value in range(300, 701, 10):
    spikes = neuron_activation.execute(leak=dac_value, timeframe=2000000)
    nrn_spike_count = [0 for nrn in halco.iter_all(NeuronOnDLS)]
    for spike in spikes:
        nrn_spike_count[int(spike.neuron)] += 1
    nrn_spike_counts.append([dac_value, nrn_spike_count])

# most certainly this can be made more beautiful
zipped = zip(*nrn_spike_counts)
#print zipped[0]
#print zipped[1]
transposed = numpy.transpose(zipped[1])

numpy.save('host_vleak_values.data', numpy.array(zipped[0]))
numpy.save('host_activity_values.data', numpy.array(transposed))

The hardware setup is shared between multiple users, we use SLURM as a resource manager and submit the experiment into a "job queue":

In [ ]:
!cat stage1_host_file*.py > stage1_host.py
!srun -p dls --daas={DLSSetup} -- singularity exec --app visionary-dls /containers/stable/latest python stage1_host.py
!ls host_*values.data*

After experiment completion, we read inb the result data files and generate a plot:

In [ ]:
%matplotlib notebook
import numpy
from matplotlib import pyplot as plt

host_vleak_values = numpy.load('host_vleak_values.data.npy')
host_activity_values = numpy.load('host_activity_values.data.npy')

for i in range(len(host_activity_values)):
    plt.plot(host_vleak_values, 1.0 * host_activity_values[i] / host_activity_values[i][-1],
             '.-', label='Neuron {}'.format(i))

plt.title("DLSv2 neuron activation measurement (Host)")
plt.ylim([-0.15, 1.15])
plt.xlabel("Leak potential [lsb]")
plt.ylabel("Activity normalized")
plt.legend(ncol=2, fontsize='xx-small')
plt.show()

## 1.2: Use on-chip processor to measure neuron spike rates

* Each neuron has a spike counter which can be read (and reset) by the embedded processor.
* The read-out has to happen before any counter saturates (255 spikes!).
* On the embedded processor: Run the read-out in a loop, accumulate into per-neuron variables.
* Transfer data to the host.
* Repeat processor for different neuron resting potentials.

In [ ]:
%%writefile stage1_ppu.py

from dlens import logger
from dlens.v2 import hal, sta, halco
from dlens.v2.halco import NeuronOnDLS, NeuronParameter, SynapseOnDLS

from tutorial_configuration import TutorialBoard, TutorialChip
logger.default_config(level=logger.LogLevel.INFO)

import numpy

def generate_poisson_spiketrain(isi, timeframe, address, synapse_driver):
    dts = numpy.random.exponential(isi, size=timeframe / isi)
    ts = numpy.add.accumulate(dts, dtype=int)
    spikes = []
    for t in ts:
        spikes.append(hal.PlaybackSpike(t, address, synapse_driver))
    return spikes

class NeuronActivation:
    def __init__(self):
        self.ctrl = sta.ExperimentControl()
        self.board = TutorialBoard()
        self.chip = TutorialChip()

    def execute(self, leak, noise_isi = 1000, timeframe = 10000000, excitatory_weight = 63, inhibitory_weight = 63):
        """
        This script measures the neuron activation of all neurons in parallel by sending excitatory and
        inhibitory Poisson spiketrains and measuring the number of neuron spikes per neuron.

        :param leak: Leak potential to set prior to sending the spiketrains
        :type leak: hal.DAC.Value
        :param noise_isi: Mean inter-spike-interval of Poisson spike sources
        :type noise_isi: int
        :param timeframe: Timeframe of spiketrains
        :type timeframe: int
        :param excitatory_weight: Weight of synaptic connection to excitatory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :param inhibitory_weight: Weight of synaptic connection to inhibitory spiketrain
        :type timeframe: hal.SynapseBlock.Synapse.Weight
        :return: Spikes received
        :rtype: list[hal.RecordedSpike]
        """

        # Logger object
        log = logger.get("execute")

        # Set leak potential
        for nrn in halco.iter_all(NeuronOnDLS):
            self.chip.capmem.set(nrn, NeuronParameter.v_leak, leak)

        synapse_address = hal.SynapseBlock.Synapse.Address(42) # arbitrary != 0

        # Configure excitatory background synapses
        for nrn in halco.iter_all(NeuronOnDLS):
            synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                     self.chip.get_excitatory_synapse_driver().toSynapseRowOnDLS())
            synapse = self.chip.get_synapse(synapse_coord)
            synapse.weight = excitatory_weight
            synapse.address = synapse_address
            self.chip.set_synapse(synapse_coord, synapse)

        # Configure inhibitory background synapses
        for nrn in halco.iter_all(NeuronOnDLS):
            synapse_coord = SynapseOnDLS(nrn.toSynapseColumnOnDLS(),
                                     self.chip.get_inhibitory_synapse_driver().toSynapseRowOnDLS())
            synapse = self.chip.get_synapse(synapse_coord)
            synapse.weight = inhibitory_weight
            synapse.address = synapse_address
            self.chip.set_synapse(synapse_coord, synapse)

        # Create a playback program (all times are in FPGA cycles / 96MHz)
        builder = hal.PlaybackProgramBuilder()

        # Create excitatory and inhibitory poisson spiketrains
        excitatory_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_excitatory_synapse_driver())
        inhibitory_spikes = generate_poisson_spiketrain(noise_isi, timeframe, synapse_address, \
            self.chip.get_inhibitory_synapse_driver())
        all_spikes = excitatory_spikes + inhibitory_spikes
        all_spikes.sort()

        # Create PPU (Plasticity Processing Unit) memory object and load pragram
        ppu_memory = hal.PPUMemory()
        ppu_memory.load_from_file("firmware/spike_counter_loop.binary")

        # Pull PPU-reset
        ppu_control_register = hal.PPUControlRegister()
        ppu_control_register.inhibit_reset = False
        builder.write(halco.PPUControlRegisterOnDLS(), ppu_control_register)

        # Write PPU memory
        builder.write(halco.PPUMemoryOnDLS(), ppu_memory)

        # Release PPU-reset
        ppu_control_register.inhibit_reset = True
        builder.write(halco.PPUControlRegisterOnDLS(), ppu_control_register)

        # Add spikes to playback program
        builder.set_time(0)
        for spike in all_spikes:
            builder.wait_until(spike.time)
            builder.fire(spike.synapse_driver, spike.source_address)
        
        # wait some additional time at the end
        builder.wait_for(10000)
        
        # Read PPU status
        ppu_status_ticket = builder.read(halco.PPUStatusRegisterOnDLS())

        # Read end-of-PPU-RAM range, where the PPU stored spike count information
        ppu_spike_count_coordinate = halco.PPUMemoryBlockOnDLS(4096 - NeuronOnDLS.size, 4095)
        ppu_spike_count_ticket = builder.read(ppu_spike_count_coordinate)

        builder.halt()
        program = builder.done()
        assert isinstance(program, hal.PlaybackProgram)

        self.ctrl.run_experiment(self.board, self.chip, program)

        ppu_status = ppu_status_ticket.get()
        # PPU finished execution
        assert (ppu_status.sleep == True)

        ppu_spike_count = ppu_spike_count_ticket.get()
        log.DEBUG([int(word.get()) for word in ppu_spike_count.words])

        spikes = program.get_spikes()
        log.TRACE("Received spikes: %s" % spikes)

        return [int(word.get()) for word in ppu_spike_count.words]


if __name__ == '__main__':
    neuron_activation = NeuronActivation()

    # Execute activation measurement for a leak potential sweep
    nrn_spike_counts = []
    for leak in range(300, 701, 10):
        nrn_spike_count = neuron_activation.execute(leak)
        nrn_spike_counts.append([leak, nrn_spike_count])

    # most certainly this can be made more beautiful
    zipped = zip(*nrn_spike_counts)
    print zipped[0]
    print zipped[1]
    transposed = numpy.transpose(zipped[1])
    
    numpy.save('ppu_vleak_values.data', numpy.array(zipped[0]))
    numpy.save('ppu_activity_values.data', numpy.array(transposed))


The C++ code for the embedded processor can be found in `spike_counter_loop.cc` (open via File dialog if interested).

The program reads out neuron-local spike counters in a loop. The per-neuron accumulation is stored to a defined memory address range after reading for a defined timeframe for readout from the host:

```cpp
// Pointer to end of heap memory
extern uint8_t ram_end;

// Spike count accumulator storage
uint32_t spike_counts[dls_num_columns]; // dls_num_columns ==  number of neurons

for (uint32_t& count: spike_counts) {
    count = 0;
}

// Timeframe of spike count measurement
time_base_t constexpr timeframe = 10000000;

// Sleep period [cycles] between consecutive readouts
uint32_t constexpr sleep_period = 10000;

// Reset counters to only measure spikes emitted during the loop below
reset_all_neuron_counters();

// Read spike count and accumulate
while (get_time_base() < timeframe) {
    for (size_t nrn = 0; nrn < dls_num_columns; ++nrn) {
        uint32_t count = get_neuron_counter(nrn);
        spike_counts[nrn] += count;
    }
    sleep_cycles(sleep_period);
}

// Write results to end of RAM
uint32_t* pos = reinterpret_cast<uint32_t*>(&ram_end) - dls_num_columns;
for (size_t i = 0; i < dls_num_columns; ++i) {
    *pos = spike_counts[i];
    pos++;
}
```

In [ ]:
!powerpc-ppu-g++ -std=gnu++14 -fdiagnostics-color=always -O2 -fno-strict-aliasing -Wall -Wextra -pedantic -ffreestanding -mcpu=nux -std=gnu++11 -fno-exceptions -fno-rtti -fno-non-call-exceptions -fno-common -ffunction-sections -fdata-sections -Ilibnux -Ilibnux -Idemo-dls/src/ppu -Idemo-dls/src/ppu -DLIBNUX_DLS_VERSION=2 -DNDEBUG -DSYSTEM_HICANN_DLS_MINI spike_counter_loop.cc -c -ospike_counter_loop.cc.7.o
!powerpc-ppu-g++ -fuse-ld=bfd -nostdlib -Lbuild/libnux -Tlibnux/libnux/elf32nux.x -Wl,--gc-sections spike_counter_loop.cc.7.o build/libnux/libnux/counter.c.10.o build/libnux/libnux/time.c.9.o build/libnux/src/crt.s.6.o build/libnux/src/cxa_pure_virtual.c.4.o build/libnux/src/initdeinit.c.3.o -ofirmware/spike_counter_loop.bin -Bstatic -Llibnux -lgcc -lnux -Bdynamic
!powerpc-ppu-objcopy -O binary firmware/spike_counter_loop.bin firmware/spike_counter_loop.binary

### Run experiment

In [ ]:
!srun -p dls --daas={DLSSetup} -- singularity exec --app visionary-dls /containers/stable/latest python stage1_ppu.py
!ls ppu*values.data*

In [ ]:
%matplotlib notebook
import numpy
from matplotlib import pyplot as plt

ppu_vleak_values = numpy.load('ppu_vleak_values.data.npy')
ppu_activity_values = numpy.load('ppu_activity_values.data.npy')

for i in range(len(ppu_activity_values)):
    plt.plot(ppu_vleak_values, 1.0 * ppu_activity_values[i] / ppu_activity_values[i][-1],
            '.-')

plt.ylim(-0.1, 1.1)
plt.title("DLSv2 neuron activation (PPU)")
plt.xlabel("Leak potential [lsb]")
plt.ylabel("Activity normalized")
plt.show()